In [55]:
import numpy as np
import matplotlib.pylab as plt
import quadprog as QP
import mpl_toolkits.mplot3d.axes3d as p3
import matplotlib.animation as ani

In [79]:
def init():  
    z.set_data(q[0,:2])
    z.set_3d_properties(q[0,2], 'z') 
    zz.set_data(qd[0,:2])
    z.set_3d_properties(qd[0,2], 'z') 
    return([z,zz]) 

In [2]:
def animate(t):      
    z.set_data(q[:t,0:2])
    z.set_3d_properties(q[:t,2], 'z') 
    zz.set_data(qd[:t,0:2])
    zz.set_3d_properties(qd[:t,2], 'z') 
    return([z,zz]) 

In [3]:
def hat_map(x):   
    return np.array([[0,-x[2],x[1]],[x[2],0,-x[0]],[-x[1],x[0],0]])

In [4]:
def position_dynamics(tau,x,xdot,v):
    return(xdot)

In [5]:
def velocity_dynamics(tau,x,xdot,v):
    v_perp = v - np.inner(x,v)*x
    return(-np.inner(xdot,xdot)*x+v_perp)

In [59]:
def reference(tau):
    ref_theta = 5*np.pi/9
    ref_phi = 0.25*tau
    return(np.array([np.sin(ref_theta)*np.cos(ref_phi),np.sin(ref_theta)*np.sin(ref_phi),np.cos(ref_theta)]))

In [60]:
def reference_velocity(tau):
    ref_theta = 5*np.pi/9
    ref_phi = 0.25*tau
    return(np.array([-0.25*np.sin(ref_theta)*np.sin(ref_phi),0.25*np.sin(ref_theta)*np.cos(ref_phi),0]))

In [61]:
def reference_acceleration(tau):
    ref_theta = 5*np.pi/9
    ref_phi = 0.25*tau
    return(np.array([-0.25**2*np.sin(ref_theta)*np.cos(ref_phi),-0.25**2*np.sin(ref_theta)*np.sin(ref_phi),0]))

In [9]:
def safety_center(tau,invariant):
    if invariant == 1:
        return(np.array([0,0,-1]))
    else:
        ref_theta = 4*np.pi/5
        ref_phi = 0.25*tau
        return(np.array([np.sin(ref_theta)*np.cos(ref_phi),np.sin(ref_theta)*np.sin(ref_phi),np.cos(ref_theta)]))

In [58]:
def safety_center_velocity(tau,invariant):  
    if invariant == 1:
        return(np.array([0,0,0]))
    else:
        ref_theta = 4*np.pi/5
        ref_phi = 0.25*tau
        return(np.array([-0.25*np.sin(ref_theta)*np.sin(ref_phi),0.25*np.sin(ref_theta)*np.cos(ref_phi),0]))

In [11]:
def safety_center_acceleration(tau,invariant):  
    if invariant == 1:
        return(np.array([0,0,0]))
    else:
        ref_theta = 4*np.pi/5
        ref_phi = 0.25*tau
        return(np.array([-0.25**2*np.sin(ref_theta)*np.cos(ref_phi),-0.25**2*np.sin(ref_theta)*np.sin(ref_phi),np.cos(ref_theta)]))

In [31]:
def position_error(tau,x):    
    return(np.dot(hat_map(x)**2,reference(tau)))

In [32]:
def velocity_error(tau,x,xdot):    
    return(xdot-np.cross(np.cross(reference(tau),reference_velocity(tau)),x))

In [13]:
def safety_constraints(tau,x,invariant,i):
    if i == 0:
        g = np.cos(np.pi/12)-1+np.inner(x,safety_center(tau,invariant))
    else:
        if i == 1:
            g = -(np.cos(np.pi/4)-1+np.inner(x,safety_center(tau,invariant)))
    return(g)

In [14]:
def CLF(tau,x,xdot):
    V1 = 0.5*np.inner(velocity_error(tau,x,xdot),velocity_error(tau,x,xdot))
    V2 = 0.5*alpha*(1-np.inner(x,reference(tau)))
    V3 = epsilon*np.inner(position_error(tau,x),velocity_error(tau,x,xdot))
    return(V1+V2+V3)

In [35]:
def h_function(tau,x,xdot,i,invariant):
    return(gamma[i]*safety_constraints(tau,x,invariant,i) + 
          ((-1)**i)*(np.inner(safety_center(tau,invariant),xdot)+np.inner(safety_center_velocity(tau,invariant),x)))

In [17]:
def CBF(tau,x,xdot,i,invariant):     
    return(1/h_function(tau,x,xdot,i,invariant))

In [20]:
def feedforward_control_law(tau,x,xdot):
    u1 = np.cross(np.cross(reference(tau),reference_acceleration(tau)),x)    
    u2 = -np.dot(hat_map(x)**2,np.cross((np.cross(reference(tau),reference_velocity(tau))),xdot).T).T
    return(u1+u2)

In [21]:
def phi_0(tau,i,invariant):    
    return((-1)**i*safety_center(tau,invariant))

In [22]:
def phi_1(tau,x,xdot,i,invariant):    
    p1 = gamma[i]*(h_function(tau,x,xdot,i,invariant)-gamma[i]*safety_constraints(tau,x,invariant,i))
    p2 = (-1)**i*(2*np.inner(xdot,safety_center_velocity(tau,invariant))+
                    np.inner(x,safety_center_acceleration(tau,invariant))-
                    np.inner(xdot,xdot)*np.inner(safety_center(tau,invariant),x))
    return(p1+p2)

In [23]:
def psi_0(tau,x,xdot):
    return(velocity_error(tau,x,xdot) + epsilon*position_error(tau,x))

In [24]:
def psi_1(tau,x,xdot):
    p11 = np.cross(np.cross(reference(tau),reference_acceleration(tau)),x)
    p12 = np.cross((np.cross(reference(tau),reference_velocity(tau))),xdot)
    p13 = np.inner(xdot,xdot)*x
    P1 = -np.inner(psi_0(tau,x,xdot),p11+p12+p13)
    P2 = -0.5*alpha*(np.inner(xdot,reference(tau))+np.inner(x,reference_velocity(tau)))
    p31 = np.cross(xdot,(np.cross(x,reference(tau))))
    p32 = np.cross(x,(np.cross(xdot,reference(tau))+np.cross(x,reference_velocity(tau))))
    P3 = np.inner(epsilon*velocity_error(tau,x,xdot),p31+p32)    
    return(P1+P2+P3)

In [40]:
def second_order_rk4_step(tau,x0,xdot0,v):    
    h = tau[1]-tau[0]    
    k11 = h*position_dynamics(tau[0],x0,xdot0,v)
    k12 = h*velocity_dynamics(tau[0],x0,xdot0,v)
    k21 = h*position_dynamics(tau[0]+h/2,x0+k11/2,xdot0+k12/2,v)
    k22 = h*velocity_dynamics(tau[0]+h/2,x0+k11/2,xdot0+k12/2,v)
    k31 = h*position_dynamics(tau[0]+h/2,x0+k21/2,xdot0+k22/2,v)
    k32 = h*velocity_dynamics(tau[0]+h/2,x0+k21/2,xdot0+k22/2,v)
    k41 = h*position_dynamics(tau[0]+h,x0+k31,xdot0+k32,v)
    k42 = h*velocity_dynamics(tau[0]+h,x0+k31,xdot0+k32,v)
    x = x0 + (k11+2*k21+2*k31+k41)/6   
    x = x/np.linalg.norm(x)
    xdot = xdot0 + (k12+2*k22+2*k32+k42)/6
    xdot = xdot - np.inner(x,xdot)*x
    return(x,xdot)

In [26]:
def leap_frog_step(tau,x0,xdot0,v):    
    dt = tau[1]-tau[0]
    if tau[0] == 0:
        x = x0 + dt*xdot0
        xdot = xdot0 + dt*velocity_dynamics(tau,x0,xdot0,v)
    else:
        x = x0 + 2*dt*xdot0
        xdot = xdot0 + 2*dt*velocity_dynamics(tau,x0,xdot0,v)
    x = x/np.linalg.norm(x)
    xdot = xdot - np.inner(x,xdot)*x
    return(x,xdot)

In [27]:
def euler_step(tau,x0,xdot0,v):    
    dt = tau[1]-tau[0]    
    x = x0 + dt*xdot0
    xdot = xdot0 + dt*velocity_dynamics(tau,x0,xdot0,v)    
    return(x,xdot)

In [28]:
def geometric_control_law(tau,x,xdot,invariant):
    G = np.concatenate((np.concatenate((H,np.zeros([3,1])),axis=1),np.array([np.array([0,0,0,Lambda])])),axis=0)
    a = np.zeros([4,])     
    c0 = np.concatenate((np.array([x]).T,np.zeros([1,1])),axis=0)
    c1 = np.concatenate((np.array([-psi_0(tau,x,xdot)]).T,np.ones([1,1])),axis=0)    
    c2 = np.concatenate((np.array([phi_0(tau,0,invariant)]).T,np.zeros([1,1])),axis=0)
    c3 = np.concatenate((np.array([phi_0(tau,1,invariant)]).T,np.zeros([1,1])),axis=0)
    C = np.concatenate((c0,c1,c2,c3),axis=1)
    u_ff = feedforward_control_law(tau,x,xdot)
    b0 = -np.inner(x,u_ff)
    b1 = np.inner(psi_0(tau,x,xdot),u_ff)+psi_1(tau,x,xdot)+eta*CLF(tau,x,xdot)    
    b2 = -(np.inner(phi_0(tau,0,invariant),u_ff)+phi_1(tau,x,xdot,0,invariant)+mu[0]*(h_function(tau,x,xdot,0,invariant))**3)
    b3 = -(np.inner(phi_0(tau,1,invariant),u_ff)+phi_1(tau,x,xdot,1,invariant)+mu[1]*(h_function(tau,x,xdot,1,invariant))**3)
    b = np.array([np.asscalar(b0),np.asscalar(b1),np.asscalar(b2),np.asscalar(b3)])    
    v,f,vu,iterations,lagrangian,iact = QP.solve_qp(G,a,C,b,1)
    v = np.array(v[0:3])
    return(v)

In [49]:
t0 = 0
tmax = 40
n_points = np.round(tmax-t0)*800
alpha = 1
epsilon = 1
gamma = [1,10]

H = np.array([[1, 0, 0],[0, 1, 0],[0,0,1]])
Lambda = 1
eta = 10
mu = 1*[1,1]
invariance = 1

In [50]:
theta0 = 1.78 
phi0 = 0
q0 = np.array([np.sin(theta0)*np.cos(phi0),np.sin(theta0)*np.sin(phi0),np.cos(theta0)])
q0 = q0/np.linalg.norm(q0)
qdot0 = np.array([0,0,0])

In [51]:
t = np.linspace(t0,tmax,n_points)
q = np.zeros([t.shape[0],3])
qdot = np.zeros([t.shape[0],3])
qd = np.zeros([t.shape[0],3])
qd_dot = np.zeros([t.shape[0],3])
g0 = np.zeros([t.shape[0],3])
g1 = np.zeros([t.shape[0],3])
h0 = np.zeros([t.shape[0],3])
h1 = np.zeros([t.shape[0],3])
V = np.zeros([t.shape[0],3])
q[0,:] = q0
qdot[0,:] = qdot0
qd[0,:] = reference(t[0])
qd_dot[0,:] = reference_velocity(t[0])
V[0] = CLF(0,q0,qdot0)
g0[0] = safety_constraints(0,q0,invariance,0)
g1[0] = safety_constraints(0,q0,invariance,1)
h0[0] = h_function(0,q0,qdot0,0,invariance)
h1[0] = h_function(0,q0,qdot0,1,invariance)

In [52]:
for i in range(1,t.size):
    u_ff = feedforward_control_law(t[i-1],q[i-1,:],qdot[i-1,:])
    u_fb = geometric_control_law(t[i-1],q[i-1,:],qdot[i-1,:],invariance)
    u = u_ff+u_fb    
    q[i,:],qdot[i,:] = euler_step(t[i-1:i+1],q[i-1,:],qdot[i-1,:],u)    
    qd[i,:] = reference(t[i])
    qd_dot[i,:] = reference_velocity(t[i])
    
    V[i] = CLF(i,q[i,:],qdot[i,:])
    g0[i] = safety_constraints(i,q[i,:],invariance,0)
    g1[i] = safety_constraints(i,q[i,:],invariance,1)
    h0[i] = h_function(i,q[i,:],qdot[i,:],0,invariance)
    h1[i] = h_function(i,q[i,:],qdot[i,:],1,invariance)

In [54]:
fig = plt.figure()
ax = p3.Axes3D(fig)
ax.plot(q[:,0], q[:,1], q[:,2])
ax.plot(qd[:,0], qd[:,1], qd[:,2])
plt.savefig("first_trajectory.pdf")
plt.close()

/usr/lib/python3/dist-packages/mpl_toolkits/mplot3d/axes3d.py:120: MatplotlibDeprecationWarning: The axesPatch function was deprecated in version 2.1. Use Axes.patch instead.
  self.axesPatch.set_linewidth(0)
/usr/lib/python3/dist-packages/mpl_toolkits/mplot3d/axes3d.py:1528: MatplotlibDeprecationWarning: The is_scalar function was deprecated in version 2.1.
  if len(args[0]) == 0 or cbook.is_scalar(args[0][0]) :
/usr/lib/python3/dist-packages/mpl_toolkits/mplot3d/axes3d.py:247: MatplotlibDeprecationWarning: The axesPatch function was deprecated in version 2.1. Use Axes.patch instead.
  self.axesPatch.draw(renderer)


In [207]:
print(qdot[-1,:])

[-0.09018386  0.26209199 -0.00239718]


In [53]:
print(np.count_nonzero(g0<0))
print(np.count_nonzero(g1<0))
print(np.count_nonzero(V<0))
print(np.count_nonzero(h0>0))
print(np.count_nonzero(h1>0))

0
0
4200
96000
96000


In [78]:
fig2 = plt.figure()
ax = p3.Axes3D(fig2)

ax.set_xlim3d([-2, 2])
ax2.set_xlabel(r'$x$', size = 9)

ax.set_ylim3d([-2, 2])
ax2.set_ylabel(r'$y$', size = 9)

ax.set_zlim3d([-2, 2])
ax2.set_zlabel(r'$z$', size = 9)

ax2.set_title(r'Simulacion de pendulo 3D', size = 11)
ax2.tick_params(labelsize = 9)
z, = ax2.plot([], [], [])
zz, = ax2.plot([], [], [])
pendulum_animation = ani.FuncAnimation(fig2, animate, init_func=init,
                               frames=q.shape[0]-10, interval=75, blit=True)
print("Guardando animacion.. (Puede tardar un par de minutos)")
pendulum_animation.save('3D-P.mp4')
plt.close()
print("Proceso terminado")

/usr/lib/python3/dist-packages/mpl_toolkits/mplot3d/axes3d.py:120: MatplotlibDeprecationWarning: The axesPatch function was deprecated in version 2.1. Use Axes.patch instead.
  self.axesPatch.set_linewidth(0)
/usr/lib/python3/dist-packages/mpl_toolkits/mplot3d/axes3d.py:247: MatplotlibDeprecationWarning: The axesPatch function was deprecated in version 2.1. Use Axes.patch instead.
  self.axesPatch.draw(renderer)


Guardando animacion.. (Puede tardar un par de minutos)


/usr/local/lib/python3.5/dist-packages/matplotlib/animation.py:1218: UserWarning: MovieWriter ffmpeg unavailable
  warnings.warn("MovieWriter %s unavailable" % writer)


ValueError: not enough values to unpack (expected 2, got 0)